# an interactive version of Tensorflow Mandelbrot set tutorial
based on original tutorial at https://www.tensorflow.org/tutorials/mandelbrot

In [1]:
# imports for simulation
import tensorflow as tf
import numpy as np

# imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display

# imports for interaction 
from ipywidgets import interact, FloatSlider
import time

In [40]:
class Timer:
    def __init__(self):
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
    def restart(self):
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
    def report(self):
        return str(round((time.perf_counter()-self.perf_start),3))
    def report_restart(self):
        result = self.report()
        self.restart()
        return result

In [45]:
def DisplayFractal(a, fmt='jpeg'):
  """given a 2D array of escape-time iteration counts, 
     render as an image"""
  a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])
  img = np.concatenate([10+20*np.cos(a_cyclic),
                        30+50*np.sin(a_cyclic),
                        155-80*np.cos(a_cyclic)], 2)
  img[a==a.max()] = 0
  a = img
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  # clear_output(wait = True)
  display(Image(data=f.getvalue()))

In [65]:
tim1 = Timer()
# numpy setup, result Z is 1D array of complex numbers
Y, X = np.mgrid[-2.0:2.0:0.005, -2.0:2.0:0.005] 
Z = X+1j*Y
print("numpy: " + tim1.report_restart())

tf.reset_default_graph()
sess = tf.InteractiveSession()
print("sess startup:", tim1.report_restart())

tzoom = tf.Variable(np.complex64(1)) # [1,1]
toffset = tf.Variable(np.complex64(-0.5+0j))  # [-.5,0]
xs = tf.Variable(Z.astype(np.complex64))
recoord = tf.Variable(xs)
zs = tf.Variable(xs)
ns = tf.Variable(tf.zeros_like(xs, tf.float32))
rezero = tf.Variable(tf.zeros_like(xs,tf.float32))

# Compute the new values of z: z^2 + x
zs_ = zs*zs + xs

# Have we diverged with this new value?
not_diverged = tf.abs(zs_) < 4

print("graph setup:", tim1.report_restart())
tf.global_variables_initializer().run()
print("graph init:", tim1.report_restart())

numpy: 0.013
sess startup: 0.001
graph setup: 2.081
graph init: 0.74


In [68]:
def recalc(zoom, xoffset, yoffset):
    tim = Timer()
    tim2 = Timer()
    #coord = (xs * tzoom) + toffset
    #zs.assign(coord)

    # Operation to re-init variables
    init = tf.group(
      ns.assign(rezero), 
      tzoom.assign(np.complex64(zoom + 1j*zoom)), 
      toffset.assign(np.complex64(xoffset + 1j*yoffset)), 
      xs.assign(recoord/tzoom + toffset), 
      # zs.assign(xs)
      zs.assign(recoord/tzoom + toffset)
    )

    # Operation to update the zs and the iteration count
    step = tf.group(
      zs.assign(zs_),
      ns.assign_add(tf.cast(not_diverged, tf.float32))
    )
    
    pre_init_time = tim.report_restart()
    
    init.run()
    init_time = tim.report_restart()
    
    for i in range(100): step.run()
    step_time = tim.report_restart()
    
    DisplayFractal(ns.eval())
    display_time = tim.report_restart()
    
    print("scale=" + str(zoom) + ", center=[" + str(xoffset) + ":" + str(yoffset) + "]")
    print("pre_init:", pre_init_time)
    print("init:", init_time)
    print("step:", step_time)
    print("display:", display_time)
    print("total recalc time:", tim2.report())

In [69]:
def zoom_changed(zoomer, xoff, yoff):
    zoom = zoomer
    recalc(zoomer, xoff, yoff)
    
interact(zoom_changed, 
         zoomer = FloatSlider(min=1,max=50,step=0.1,value=1, continuous_update=False), 
         xoff = FloatSlider(min=-2, max=2, step=0.05, value=-0.5, continuous_update=False), 
         yoff = FloatSlider(min=-2, max=2, step=0.05, value=0.0, continuous_update=False))

A Jupyter Widget

<function __main__.zoom_changed>

In [6]:
# sess.close()